# 기본 웹 크롤링 코드 정리

In [49]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import NoSuchElementException 

import time
import re
from bs4 import BeautifulSoup 
from tqdm import tqdm 


- 공공 데이터 포털의 서울시 식당 데이터 이용

https://data.seoul.go.kr/dataList/OA-16094/S/1/datasetView.do

In [ ]:
# 데이터 읽기
df = pd.read_csv('./seoul_res_data.csv',encoding='cp949')
df['naver_map_url'] = '' # 미리 url을 담을 column을 만들어줌 

driver = webdriver.Chrome(executable_path='C:/Users/ysj_1/Documents/ds_study/driver/chromedriver.exe') # 웹드라이버가 설치된 경로를 지정해주시면 됩니다.

C:\Users\ysj_1\AppData\Local\Temp\ipykernel_22844\3121499545.py:1: DtypeWarning: Columns (2,8,13,14,25,27,28,29,30,36,39,41,42,46,48,56,57,59,60,61,62,63,66,71,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./seoul_res_data.csv',encoding='cp949')
C:\Users\ysj_1\AppData\Local\Temp\ipykernel_22844\3121499545.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='C:/Users/ysj_1/Documents/ds_study/driver/chromedriver.exe') # 웹드라이버가 설치된 경로를 지정해주시면 됩니다.


In [51]:
# 사용컬럼 정리리
df = df[['영업상태코드', '영업상태명', '상세영업상태코드',
            '상세영업상태명', '전화번호', '지번주소', '도로명주소', '도로명우편번호', '사업장명', '최종수정일자', '데이터갱신구분',
            '데이터갱신일자', '업태구분명', '좌표정보(X)', '좌표정보(Y)', '홈페이지','naver_map_url']]

In [52]:

for i,v in enumerate(df['지번주소']):  # df['지번주소'] 열의 각 행에 대해 인덱스와 함께 반복문 실행
    if type(v) == float:  # 만약 해당 행의 값이 float 타입이면
        print(i,v)  # 해당 행의 인덱스와 값을 출력

4553 nan
4562 nan
5925 nan
10804 nan
10832 nan
10844 nan
12100 nan
12305 nan
15268 nan
15843 nan
16041 nan
16093 nan
18005 nan
20495 nan
20496 nan
21657 nan
21673 nan
21771 nan
21772 nan
21773 nan
21774 nan
21775 nan
24131 nan
30303 nan
33204 nan
33520 nan
34261 nan
34600 nan
34769 nan
35300 nan
37000 nan
37931 nan
38837 nan
39135 nan
39361 nan
45124 nan
48768 nan
50680 nan
50997 nan
52426 nan
53335 nan
57089 nan
57321 nan
78989 nan
79029 nan
79054 nan
79072 nan
79324 nan
86226 nan
98811 nan
114228 nan
114347 nan
131933 nan
172560 nan
172796 nan
173212 nan
173517 nan
174588 nan
174623 nan
175577 nan
178924 nan
178988 nan
179376 nan
179971 nan
180472 nan
182248 nan
182653 nan
182759 nan
182788 nan
191119 nan
199735 nan
199847 nan
200031 nan
200673 nan
202756 nan
202843 nan
203139 nan
205410 nan
205467 nan
205471 nan
205476 nan
205498 nan
205663 nan
205747 nan
205768 nan
205775 nan
205821 nan
205826 nan
205841 nan
205849 nan
205866 nan
205911 nan
205914 nan
205925 nan
205926 nan
205935 n

In [53]:
df['지번주소'].fillna('서울 서울 서울',inplace=True) # '지번주소' 컬럼의 결측치를 '서울 서울 서울'로 대체
for i,v in enumerate(df['지번주소']): # '지번주소' 컬럼의 각 행에 대해 반복문 실행
    if type(v) == float: # 만약 해당 행의 값이 float 타입이면
        print(i,v) # 해당 행의 인덱스와 값을 출력

In [54]:
# 지번주소 컬럼에서 문자열을 공백으로 분리한 후 길이가 3보다 작은 경우를 필터링하여 해당 행을 제외한 데이터프레임을 새로 생성합니다.
df = df[~(df['지번주소'].apply(lambda x: len(str(x).split(' '))<3))]

In [55]:
# '지번주소' 컬럼에서 띄어쓰기를 기준으로 분리하여 '구' 컬럼에 저장
df['구'] = df['지번주소'].apply(lambda x: (str(x).split(' '))[1])

# '지번주소' 컬럼에서 띄어쓰기를 기준으로 분리하여 '동' 컬럼에 저장
df['동'] = df['지번주소'].apply(lambda x: (str(x).split(' '))[2])

# '구'와 '동' 컬럼만 선택하여 출력
df[['구','동']]

,구,동
0,강남구,역삼동
1,강남구,청담동
2,강남구,역삼동
3,송파구,가락동
4,송파구,방이동
...,...,...
490077,용산구,한강로2가
490078,양천구,신월동
490079,종로구,명륜3가
490080,서대문구,창천동


In [56]:

# '구', '동', '사업장명' 컬럼을 합쳐서 '검색어' 컬럼 생성
df['검색어'] = (df['구'] + ' ' + df['동'] + ' ' + df['사업장명'])
df  # 결과 출력


,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,전화번호,지번주소,도로명주소,도로명우편번호,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),홈페이지,naver_map_url,구,동,검색어
0,1,영업/정상,1,영업,NaN,서울특별시 강남구 역삼동 823 삼원타워,"서울특별시 강남구 테헤란로 124, 삼원타워 지하1층 123호 (역삼동)",6234.0,랩보이(wrap boy),2023-03-31 15:18,I,2022-12-04 0:02,기타,202722.5318,444055.0707,NaN,,강남구,역삼동,강남구 역삼동 랩보이(wrap boy)
1,1,영업/정상,1,영업,NaN,서울특별시 강남구 청담동 46-11 근정 빌딩,"서울특별시 강남구 삼성로133길 15, 근정 빌딩 지하1층 (청담동)",6068.0,은가비,2023-03-31 15:50,I,2022-12-04 0:02,경양식,204228.2373,446332.6927,NaN,,강남구,청담동,강남구 청담동 은가비
2,1,영업/정상,1,영업,NaN,서울특별시 강남구 역삼동 711-6 역삼자이,"서울특별시 강남구 언주로 408, 역삼자이 상가동 지하1층 B10호 (역삼동)",6214.0,청춘부추소곱창,2023-03-31 16:41,I,2022-12-04 0:02,한식,203882.7780,444136.1670,NaN,,강남구,역삼동,강남구 역삼동 청춘부추소곱창
3,1,영업/정상,1,영업,02 448 6648,서울특별시 송파구 가락동 84-6,"서울특별시 송파구 송이로20길 12-1, 1층 101호 (가락동)",5712.0,치카바,2023-05-12 14:40,I,2022-12-04 23:04,일식,210536.3795,443748.6844,NaN,,송파구,가락동,송파구 가락동 치카바
4,1,영업/정상,1,영업,NaN,서울특별시 송파구 방이동 57 One57빌당,"서울특별시 송파구 올림픽로34길 27-23, One57빌딩 2층 201호 (방이동)",5547.0,오고보,2023-03-31 9:55,I,2022-12-04 0:02,경양식,209835.4199,445884.5368,NaN,,송파구,방이동,송파구 방이동 오고보
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490077,1,영업/정상,1,영업,02 7924157,서울특별시 용산구 한강로2가 116-1 1층,"서울특별시 용산구 한강대로 130-3, 1층 (한강로2가)",4382.0,메가엠지씨커피 신용산역점,2022-05-13 13:45,U,2021-12-04 23:05,기타,197277.8630,447641.4505,NaN,,용산구,한강로2가,용산구 한강로2가 메가엠지씨커피 신용산역점
490078,3,폐업,2,폐업,226989233,서울특별시 양천구 신월동 146-3,서울특별시 양천구 곰달래로5길 85 (신월동),7920.0,황제꼼장어,2022-05-13 13:43,U,2021-12-04 23:05,기타,184835.9729,448165.3497,NaN,,양천구,신월동,양천구 신월동 황제꼼장어
490079,1,영업/정상,1,영업,NaN,서울특별시 종로구 명륜3가 1-14,서울특별시 종로구 명륜길 31 (명륜3가),3071.0,희래등반점,2022-06-29 9:19,U,2021-12-07 0:01,중국식,199506.1836,454029.3973,NaN,,종로구,명륜3가,종로구 명륜3가 희래등반점
490080,3,폐업,2,폐업,02 362 6238,서울특별시 서대문구 창천동 2-19 (지상1층),"서울특별시 서대문구 연세로12길 28 (창천동,(지상1층))",3776.0,면가요시,2022-06-21 13:27,U,2021-12-05 22:03,한식,194500.2851,450742.0375,NaN,,서대문구,창천동,서대문구 창천동 면가요시


In [57]:
# '상세영업상태명'이 '영업'인 행만 추출하여 새로운 데이터프레임 생성
df = df[df['상세영업상태명'] == '영업'].reset_index(drop=True)


In [60]:

# 데이터프레임의 '검색어' 열에서 키워드를 하나씩 가져온다.
for i, keyword in enumerate(df['검색어'].tolist()):

    # 현재 몇 번째 행의 키워드를 찾고 있는지 출력한다.
    print("이번에 찾을 키워드 :", i+1, f"/ {df.shape[0]} 행", keyword)

    try:  # 예외처리 시작
        # 검색 url 만들기
        naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place'
        driver.get(naver_map_search_url)  # 검색 url 접속 = 검색하기
        time.sleep(4)  # 중요

        cu = driver.current_url  # 검색이 성공된 플레이스에 대한 개별 페이지
        res_code = re.findall(r"place/(\d+)", cu)  # 정규표현식으로 플레이스 코드 추출
        final_url = 'https://pcmap.place.naver.com/restaurant/' + \
            res_code[0]+'/review/visitor#'  # 리뷰 페이지 url 만들기

        print(final_url)  # 리뷰 페이지 url 출력
        # 데이터프레임의 'naver_map_url' 열에 리뷰 페이지 url 저장
        df['naver_map_url'][i] = final_url

    except IndexError:  # 만약 플레이스 코드를 찾을 수 없다면
        df['naver_map_url'][i] = ''  # 데이터프레임의 'naver_map_url' 열에 빈 문자열 저장
        print('none')  # 'none' 출력

    # 데이터프레임을 csv 파일로 저장한다.
    df.to_csv('url_completed.csv', encoding='utf-8-sig')


이번에 찾을 키워드 : 1 / 30 행 구로구 구로동 청춘국물닭발
https://pcmap.place.naver.com/restaurant/1489298006/review/visitor#
이번에 찾을 키워드 : 2 / 30 행 중구 광희동1가 나란


C:\Users\ysj_1\AppData\Local\Temp\ipykernel_22844\1315469651.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['naver_map_url'][i] = ''  # 데이터프레임의 'naver_map_url' 열에 빈 문자열 저장


none
이번에 찾을 키워드 : 3 / 30 행 강동구 길동 쯔끼
https://pcmap.place.naver.com/restaurant/37486780/review/visitor#
이번에 찾을 키워드 : 4 / 30 행 영등포구 여의도동 천지면옥


C:\Users\ysj_1\AppData\Local\Temp\ipykernel_22844\1315469651.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['naver_map_url'][i] = final_url


https://pcmap.place.naver.com/restaurant/1794894008/review/visitor#
이번에 찾을 키워드 : 5 / 30 행 송파구 잠실동 이노시시
https://pcmap.place.naver.com/restaurant/1607655671/review/visitor#
이번에 찾을 키워드 : 6 / 30 행 영등포구 대림동 정호프
https://pcmap.place.naver.com/restaurant/1887825345/review/visitor#
이번에 찾을 키워드 : 7 / 30 행 강남구 개포동 청년피자 개포점
none
이번에 찾을 키워드 : 8 / 30 행 구로구 신도림동 오봉집 신도림점
none
이번에 찾을 키워드 : 9 / 30 행 영등포구 대림동 미쳐버린파닭(대림점)
none
이번에 찾을 키워드 : 10 / 30 행 송파구 장지동 교촌치킨 위례송파점
none
이번에 찾을 키워드 : 11 / 30 행 마포구 상암동 중화백반
https://pcmap.place.naver.com/restaurant/1808824434/review/visitor#
이번에 찾을 키워드 : 12 / 30 행 중구 을지로6가 이찬스 타쿠미주방 동대문점
none
이번에 찾을 키워드 : 13 / 30 행 광진구 구의동 쿠마이누
https://pcmap.place.naver.com/restaurant/37638629/review/visitor#
이번에 찾을 키워드 : 14 / 30 행 강서구 화곡동 맘모스치킨
none
이번에 찾을 키워드 : 15 / 30 행 강동구 성내동 하다식당
none
이번에 찾을 키워드 : 16 / 30 행 광진구 자양동 고래맥주창고(뚝섬유원지지점)
https://pcmap.place.naver.com/restaurant/1493006463/review/visitor#
이번에 찾을 키워드 : 17 / 30 행 구로구 구로동 티엠비 숯불바베큐 구디점
https://pcmap.place.naver.com/restaurant/

In [61]:
driver.quit()

In [62]:
df['naver_map_url']

0     https://pcmap.place.naver.com/restaurant/14892...
1                                                      
2     https://pcmap.place.naver.com/restaurant/37486...
3     https://pcmap.place.naver.com/restaurant/17948...
4     https://pcmap.place.naver.com/restaurant/16076...
5     https://pcmap.place.naver.com/restaurant/18878...
6                                                      
7                                                      
8                                                      
9                                                      
10    https://pcmap.place.naver.com/restaurant/18088...
11                                                     
12    https://pcmap.place.naver.com/restaurant/37638...
13                                                     
14                                                     
15    https://pcmap.place.naver.com/restaurant/14930...
16    https://pcmap.place.naver.com/restaurant/17774...
17                                              

In [63]:
import pandas as pd 
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

import time
import re 
from bs4 import BeautifulSoup 
from tqdm import tqdm 

# 웹드라이버 접속
driver = webdriver.Chrome(executable_path='C:/Users/ysj_1/Documents/ds_study/driver/chromedriver.exe')

# # 전처리 완료한 데이터 불러오기
# # url이 없는 경우는 제거함
# df = pd.read_csv('url_completed.csv') 

# 수집할 정보들 
rating_list = []    # rating
user_review_id = {} # user id
review_json = {} # 리뷰 
image_json = {} # 리뷰 이미지

C:\Users\ysj_1\AppData\Local\Temp\ipykernel_22844\1259476475.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='C:/Users/ysj_1/Documents/ds_study/driver/chromedriver.exe')


In [64]:
df_url = df[df['naver_map_url'] != '' ]

In [65]:
df_url.reset_index(drop=True,inplace=True)
df_url.head(3)

,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,전화번호,지번주소,도로명주소,도로명우편번호,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),홈페이지,naver_map_url,구,동,검색어
0,1,영업/정상,1,영업,NaN,서울특별시 구로구 구로동 545-5 1층,"서울특별시 구로구 공원로9길 10, 1층 (구로동)",8290.0,청춘국물닭발,2022-05-13 15:56,U,2021-12-04 23:05,한식,189991.0159,444544.5268,NaN,https://pcmap.place.naver.com/restaurant/14892...,구로구,구로동,구로구 구로동 청춘국물닭발
1,1,영업/정상,1,영업,NaN,서울특별시 강동구 길동 333-7 에이프러스빌,"서울특별시 강동구 양재대로124길 29, 1층 101호 (길동, 에이프러스빌)",5302.0,쯔끼,2022-05-13 11:32,U,2021-12-04 23:05,정종/대포집/소주방,212639.6699,448977.6940,NaN,https://pcmap.place.naver.com/restaurant/37486...,강동구,길동,강동구 길동 쯔끼
2,1,영업/정상,1,영업,NaN,"서울특별시 영등포구 여의도동 13 여의도파라곤 101,1호","서울특별시 영등포구 국회대로 800, 여의도파라곤 지상1층 101,102,103호 ...",7238.0,천지면옥,2022-05-13 10:50,U,2021-12-04 23:05,냉면집,192959.2712,447637.6561,NaN,https://pcmap.place.naver.com/restaurant/17948...,영등포구,여의도동,영등포구 여의도동 천지면옥


In [67]:
# 지정된 url로 이동하여 리뷰를 크롤링하는 코드

for i in range(len(df_url)): 
    print('총 식당 개수 : ',len(df_url))
    print('======================================================') 
    print(str(i+1)+'번째 식당') 
    
    
    # 식당 리뷰 개별 url 접속
    driver.get(df_url['naver_map_url'][i]) 
    thisurl = df_url['naver_map_url'][i]
    time.sleep(2) 
    
    # 더보기 버튼 다 누를 것
    # 더보기 버튼은 10개 마다 나옴
    while True: 
        try: 
            time.sleep(1) 
            driver.find_element_by_tag_name('body').send_keys(Keys.END) 
            time.sleep(1) 
            
            driver.find_element_by_css_selector('#app-root > div > div > div > div:nth-child(7) > div:nth-child(3) > div:nth-child(3) > div.lfH3O > a').click()
            time.sleep(1) 
            driver.find_element_by_tag_name('body').send_keys(Keys.END) 
             
            
        except NoSuchElementException: 
            print('-더보기 버튼 모두 클릭 완료-') 
            break 
    

    # 파싱
    html = driver.page_source 
    soup = BeautifulSoup(html, 'lxml') 
    time.sleep(1) 
    
    # 식당 구분 
    restaurant_name = df_url['검색어'][i]
    print('식당 이름 : '+restaurant_name) 


    # 특정 식당에 대한 리뷰 수집
    try: 
        one_review = soup.find_all('li', attrs = {'class':'YeINN'})
        review_num = len(one_review) # 특정 식당의 리뷰 총 개수 
        print('리뷰 총 개수 : '+str(review_num)) 
        
        # 모든 리뷰에 대해서 정보 수집
        for i in range(len(one_review)): 
            
            
            # rating, 별점 
            try:
                rating = rating = one_review[i].find('span', attrs = {'class':'P1zUJ HNG_1'}).text
            except:
                rating = '별점없음'
            print('rating = '+rating) 
            
     
             
            # date 
            # 사진 리뷰 없음 
            if len(one_review[i].find_all('span', attrs = {'class':'P1zUJ'})) == 5: 
                date = one_review[i].find_all('span', attrs = {'class':'P1zUJ'})[2].text 

            elif len(one_review[i].find_all('span', attrs = {'class':'P1zUJ'})) == 6: 
                date = one_review[i].find_all('span', attrs = {'class':'P1zUJ'})[3].text
            else: 
                date = ""
            print('date = '+date)

            
            # review 내용
            try: 
                review_content = one_review[i].find('span', attrs = {'class':'zPfVt'}).text
            except: # 리뷰가 없다면
                review_content = "" 
            print('리뷰 내용 : '+review_content) 
            
            
            print('----------------------------------------------') 
            print('\n') 
            

            naver_review = {'restaurant_name' : restaurant_name,'review_content' : review_content, 'rating' : rating, 'date' : date}
            rating_list.append(naver_review) 

    # 리뷰가 없는 경우        
    except NoSuchElementException: 
        none_review = "네이버 리뷰 없음" 
        print(none_review)
        review_num = 0 
    
        naver_review = {'restaurant_name' : restaurant_name,'review_content' : none_review, 'rating' : none_review, 'date' : none_review}
        rating_list.append(naver_review)

print('\n')

총 식당 개수 :  11
0번째 식당


C:\Users\ysj_1\AppData\Local\Temp\ipykernel_22844\2767064793.py:17: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  driver.find_element_by_tag_name('body').send_keys(Keys.END)
C:\Users\ysj_1\AppData\Local\Temp\ipykernel_22844\2767064793.py:20: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector('#app-root > div > div > div > div:nth-child(7) > div:nth-child(3) > div:nth-child(3) > div.lfH3O > a').click()


-더보기 버튼 모두 클릭 완료-
식당 이름 : 구로구 구로동 청춘국물닭발
리뷰 총 개수 : 3
rating = 별점없음
date = 최근 방문일22.8.3.수2022년 8월 3일 수요일
리뷰 내용 : 
----------------------------------------------


rating = 별점없음
date = 최근 방문일22.8.3.수2022년 8월 3일 수요일
리뷰 내용 : 
----------------------------------------------


rating = 별점없음
date = 최근 방문일22.8.1.월2022년 8월 1일 월요일
리뷰 내용 : 
----------------------------------------------


총 식당 개수 :  11
1번째 식당


C:\Users\ysj_1\AppData\Local\Temp\ipykernel_22844\2767064793.py:22: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  driver.find_element_by_tag_name('body').send_keys(Keys.END)


-더보기 버튼 모두 클릭 완료-
식당 이름 : 강동구 길동 쯔끼
리뷰 총 개수 : 54
rating = 별점없음
date = 최근 방문일2.28.화2023년 2월 28일 화요일
리뷰 내용 : 메뉴 도장깨기 하는중입니다 ,, 😝 짬뽕탕, 오뎅탕은 지난번에 먹어봐서 오늘은 차돌숙주볶음을 먹어봤는데 고기가 아주 큼지막하게 들어가 있어요 ! 숙주도 아삭아삭하니 맛있었어요 🤤  
지난번에 먹고 최애메뉴된 오다리튀김 다시 주문했어요 ㅎㅎ 바삭바삭하고 소스 찍어먹으면 더 맛있더라구요 ,, 튀김류는 꼭 한 종류 주문해서 맛보는걸 추천드립니다 !!  감자 고로케도 부
드러워서 가볍게 먹기 괜찮아요 ,, 다음번에 또 방문할게요 ✌🏻
----------------------------------------------


rating = 별점없음
date = 최근 방문일3.25.토2023년 3월 25일 토요일
리뷰 내용 : 사장님도 너무 친절하시고 무엇보다... 안주 무엇????? 진짜맛있어서ㅠ 사케가 술술 넘어가요
내돈내산 남편이랑 데이트하기 짱좋음❤️
----------------------------------------------


rating = 별점없음
date = 최근 방문일4.10.월2023년 4월 10일 월요일
리뷰 내용 : 음식이 맛있어서 순삭했어요ㅎㅎ 닭튀김 추천드립니당! 튀김 요리 맥주에 너무 잘 어울려용 강동구 이자카야 추천드립니다!!
----------------------------------------------


rating = 별점없음
date = 최근 방문일1.27.금2023년 1월 27일 금요일
리뷰 내용 : 집 근처에 생긴 이자카야 ! 처음이라 이것저것 시켜서 먹어보았어요 😝 오뎅탕 가격에 비해 엄청 푸짐해요 ,, 어묵이 종류별로 들어있어 골라먹는 재미가 있더라구요 ! 그리고 최애메뉴된 오
다리튀김 🫶🏻 바삭바삭하고 맛은 후라이드치킨 같은 맛,,  오징어튀김 잘 안 먹는데도 넘 맛있었어요 다음번에도 또 올거예요 완전 추천 !!
-------

In [68]:
driver.quit()

In [69]:
review_df = pd.DataFrame(rating_list)

In [70]:
review_df

,restaurant_name,review_content,rating,date
0,구로구 구로동 청춘국물닭발,,별점없음,최근 방문일22.8.3.수2022년 8월 3일 수요일
1,구로구 구로동 청춘국물닭발,,별점없음,최근 방문일22.8.3.수2022년 8월 3일 수요일
2,구로구 구로동 청춘국물닭발,,별점없음,최근 방문일22.8.1.월2022년 8월 1일 월요일
3,강동구 길동 쯔끼,"메뉴 도장깨기 하는중입니다 ,, 😝 짬뽕탕, 오뎅탕은 지난번에 먹어봐서 오늘은 차돌...",별점없음,최근 방문일2.28.화2023년 2월 28일 화요일
4,강동구 길동 쯔끼,사장님도 너무 친절하시고 무엇보다... 안주 무엇????? 진짜맛있어서ㅠ 사케가 술...,별점없음,최근 방문일3.25.토2023년 3월 25일 토요일
...,...,...,...,...
697,종로구 명륜3가 희래등반점,친구들이랑 같이 먹을수 있는집 찾다가 그냥 들어와밨는데 너무 맛있네요~ 즐거운 시간...,별점없음,최근 방문일22.7.25.월2022년 7월 25일 월요일
698,종로구 명륜3가 희래등반점,,별점없음,최근 방문일22.7.20.수2022년 7월 20일 수요일
699,종로구 명륜3가 희래등반점,굿,별점없음,최근 방문일22.7.3.일2022년 7월 3일 일요일
700,종로구 명륜3가 희래등반점,굿,별점없음,최근 방문일21.12.21.화2021년 12월 21일 화요일


In [72]:
review_df['rating'].value_counts()

별점없음     584
별점5       46
별점4       39
별점4.5     10
별점3        9
별점3.5      9
별점1        2
별점2.5      1
별점0.5      1
별점1.5      1
Name: rating, dtype: int64

In [48]:
driver.quit()

# URL 추출

In [73]:
from tqdm import tqdm  # tqdm 라이브러리 불러오기
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import NoSuchElementException 

import time
import re
from bs4 import BeautifulSoup 
from tqdm import tqdm 


df = pd.read_csv('./seoul_res_data.csv',encoding='cp949')
df['naver_map_url'] = '' # 미리 url을 담을 column을 만들어줌 

driver = webdriver.Chrome(executable_path='C:/Users/ysj_1/Documents/ds_study/driver/chromedriver.exe') # 웹드라이버가 설치된 경로를 지정해주시면 됩니다.

df = df[['영업상태코드', '영업상태명', '상세영업상태코드',
            '상세영업상태명', '전화번호', '지번주소', '도로명주소', '도로명우편번호', '사업장명', '최종수정일자', '데이터갱신구분',
            '데이터갱신일자', '업태구분명', '좌표정보(X)', '좌표정보(Y)', '홈페이지','naver_map_url']]


df['지번주소'].fillna('서울 서울 서울',inplace=True) # '지번주소' 컬럼의 결측치를 '서울 서울 서울'로 대체


# 지번주소 컬럼에서 문자열을 공백으로 분리한 후 길이가 3보다 작은 경우를 필터링하여 해당 행을 제외한 데이터프레임을 새로 생성합니다.
df = df[~(df['지번주소'].apply(lambda x: len(str(x).split(' '))<3))]


# '지번주소' 컬럼에서 띄어쓰기를 기준으로 분리하여 '구' 컬럼에 저장
df['구'] = df['지번주소'].apply(lambda x: (str(x).split(' '))[1])
# '지번주소' 컬럼에서 띄어쓰기를 기준으로 분리하여 '동' 컬럼에 저장
df['동'] = df['지번주소'].apply(lambda x: (str(x).split(' '))[2])


# '구', '동', '사업장명' 컬럼을 합쳐서 '검색어' 컬럼 생성
df['검색어'] = (df['구'] + ' ' + df['동'] + ' ' + df['사업장명'])


# '상세영업상태명'이 '영업'인 행만 추출하여 새로운 데이터프레임 생성
df = df[df['상세영업상태명'] == '영업'].reset_index(drop=True)

# df 일부만 추출
df = df.tail(30) 
df.reset_index(drop=True,inplace=True)


# 데이터프레임의 '검색어' 열에서 키워드를 하나씩 가져와서 반복한다.
for i, keyword in tqdm(enumerate(df['검색어'].tolist())):

    # 현재 몇 번째 행의 키워드를 찾고 있는지 출력한다.
    print("이번에 찾을 키워드 :", i+1, f"/ {df.shape[0]} 행", keyword)

    try:  # 예외처리 시작
        # 검색 url 만들기
        naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place'
        driver.get(naver_map_search_url)  # 검색 url 접속 = 검색하기
        time.sleep(4)  # 중요

        cu = driver.current_url  # 검색이 성공된 플레이스에 대한 개별 페이지
        res_code = re.findall(r"place/(\d+)", cu)  # 정규표현식으로 플레이스 코드 추출
        final_url = 'https://pcmap.place.naver.com/restaurant/' + \
            res_code[0]+'/review/visitor#'  # 리뷰 페이지 url 만들기

        print(final_url)  # 리뷰 페이지 url 출력
        # 데이터프레임의 'naver_map_url' 열에 리뷰 페이지 url 저장
        df['naver_map_url'][i] = final_url

    except IndexError:  # 만약 플레이스 코드를 찾을 수 없다면
        df['naver_map_url'][i] = ''  # 데이터프레임의 'naver_map_url' 열에 빈 문자열 저장
        print('none')  # 'none' 출력

    # 데이터프레임을 csv 파일로 저장한다.
    df.to_csv('url_completed.csv', encoding='utf-8-sig')

driver.quit()

C:\Users\ysj_1\AppData\Local\Temp\ipykernel_22844\2909048714.py:13: DtypeWarning: Columns (2,8,13,14,25,27,28,29,30,36,39,41,42,46,48,56,57,59,60,61,62,63,66,71,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./seoul_res_data.csv',encoding='cp949')
C:\Users\ysj_1\AppData\Local\Temp\ipykernel_22844\2909048714.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='C:/Users/ysj_1/Documents/ds_study/driver/chromedriver.exe') # 웹드라이버가 설치된 경로를 지정해주시면 됩니다.
0it [00:00, ?it/s]

이번에 찾을 키워드 : 1 / 30 행 구로구 구로동 청춘국물닭발


1it [00:07,  7.38s/it]

https://pcmap.place.naver.com/restaurant/1489298006/review/visitor#
이번에 찾을 키워드 : 2 / 30 행 중구 광희동1가 나란


C:\Users\ysj_1\AppData\Local\Temp\ipykernel_22844\2909048714.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['naver_map_url'][i] = ''  # 데이터프레임의 'naver_map_url' 열에 빈 문자열 저장
2it [00:11,  5.57s/it]

...

none
이번에 찾을 키워드 : 29 / 30 행 용산구 한강로2가 메가엠지씨커피 신용산역점


29it [02:06,  4.23s/it]

none
이번에 찾을 키워드 : 30 / 30 행 종로구 명륜3가 희래등반점


30it [02:10,  4.34s/it]

https://pcmap.place.naver.com/restaurant/38563503/review/visitor#


# 리뷰 크롤링

In [ ]:
import pandas as pd 
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

import time
import re 
from bs4 import BeautifulSoup 
from tqdm import tqdm 

# 웹드라이버 접속
driver = webdriver.Chrome(executable_path='C:/Users/ysj_1/Documents/ds_study/driver/chromedriver.exe')

# # 전처리 완료한 데이터 불러오기
# # url이 없는 경우는 제거함
# df = pd.read_csv('url_completed.csv') 

# 수집할 정보들 
rating_list = []    # rating


# naver_map_url 열이 비어있지 않은 행만 추출하여 df_url에 저장
df_url = df[df['naver_map_url'] != '' ]
# 추출된 행들의 인덱스를 재설정
df_url.reset_index(drop=True,inplace=True)



for i in tqdm(range(len(df_url))): 
    print('총 식당 개수 : ',len(df_url))
    print('======================================================') 
    print(str(i+1)+'번째 식당') 
    
    
    # 식당 리뷰 개별 url 접속
    driver.get(df_url['naver_map_url'][i]) 
    thisurl = df_url['naver_map_url'][i]
    time.sleep(2) 
    
    # 더보기 버튼 다 누를 것
    # 더보기 버튼은 10개 마다 나옴
    while True: 
        try: 
            time.sleep(1) 
            driver.find_element_by_tag_name('body').send_keys(Keys.END) 
            time.sleep(1) 
            
            driver.find_element_by_css_selector('#app-root > div > div > div > div:nth-child(7) > div:nth-child(3) > div:nth-child(3) > div.lfH3O > a').click()
            time.sleep(1) 
            driver.find_element_by_tag_name('body').send_keys(Keys.END) 
             
            
        except NoSuchElementException: 
            print('-더보기 버튼 모두 클릭 완료-') 
            break 
    

    # 파싱
    html = driver.page_source 
    soup = BeautifulSoup(html, 'lxml') 
    time.sleep(1) 
    
    # 식당 구분 
    restaurant_name = df_url['검색어'][i]
    print('식당 이름 : '+restaurant_name) 
    


    # 특정 식당에 대한 리뷰 수집
    try: 
        one_review = soup.find_all('li', attrs = {'class':'YeINN'})
        review_num = len(one_review) # 특정 식당의 리뷰 총 개수 
        print('리뷰 총 개수 : '+str(review_num)) 
        
        # 모든 리뷰에 대해서 정보 수집
        for i in range(len(one_review)): 
            
            # rating, 별점 
            try:
                rating = rating = one_review[i].find('span', attrs = {'class':'P1zUJ HNG_1'}).text
            except:
                rating = '별점없음'
            print('rating = '+rating) 
    
            # date 
            # 사진 리뷰 없음 
            if len(one_review[i].find_all('span', attrs = {'class':'P1zUJ'})) == 5: 
                date = one_review[i].find_all('span', attrs = {'class':'P1zUJ'})[2].text 

            elif len(one_review[i].find_all('span', attrs = {'class':'P1zUJ'})) == 6: 
                date = one_review[i].find_all('span', attrs = {'class':'P1zUJ'})[3].text
            else: 
                date = ""
            print('date = '+date)

            
            # review 내용
            try: 
                review_content = one_review[i].find('span', attrs = {'class':'zPfVt'}).text
            except: # 리뷰가 없다면
                review_content = "" 
            print('리뷰 내용 : '+review_content) 
            
            print('----------------------------------------------') 
            print('\n') 
            
            
            # rating_df_url_list 
            naver_review = {'restaurant_name' : restaurant_name,'review_content' : review_content, 'rating' : rating, 'date' : date}
            rating_list.append(naver_review) 

    # 리뷰가 없는 경우        
    except NoSuchElementException: 
        none_review = "네이버 리뷰 없음" 
        print(none_review)
        review_num = 0 

        # rating_df_url_list
        naver_review = {'restaurant_name' : restaurant_name,'review_content' : none_review, 'rating' : none_review, 'date' : none_review}
        rating_list.append(naver_review)

print('\n')
driver.quit()


In [ ]:
review_df = pd.DataFrame(rating_list)
review_df